# Wearable Device data preprocessing

## Explanations on Device and Data Variables

* Wearable Device = Samsung Galaxy Watch 4 Classic 
* ~.device_profile = Device information and ID.
* ~.ecg = Electrocardiogram value. It only contains mean heart rate value.
* ~.floors_climbed = Number of floors you've climbed. (Inaccurate)
* ~.food_info = Information of the food you take. (Inaccurate)
* ~.food_intake = Food intake information including variables like calories. (Inaccurate)
* ~.height =  User's height. User should put the information in samsung health application.
* ~.nutrition = Food intake nutrition information. (Inaccurate)
* ~.sleep_stage = Sleep stages including REM sleep. Automatically calculates when user sleep with their watch on. 
* ~.user_profile = User profiles like birth, nationality, etc. User should put the information.
* ~.water_intake = Glass of water intake. User should put the information everytime.
* ~.weight = User's weight. User should put the information.
* ~.activity.day_summary = Summary of user's activity information. Contains step count, active time, distance, run time, and walk time.
* ~.activity.goal = Not sure.
* ~.activity.level = Not sure.
* ~.alered_heart_rate = Not sure. Maybe alerted history, when user's heart rate is unusually high or low.
* ~.best_records = Not sure.
* ~.breathing = Breathing training history.
* ~.calories_burned.details = Not sure.
* ~.exercise = User's exercise information. Contains exercise distance, calories, etc.
* ~.exercise.weather = Not sure.
* ~.food_frequent = Not sure.
* ~.goal_history = Not sure.
* ~.insight.milestones = Not sure.
* ~.permission = Not sure.
* ~.preferences = Not sure.
* ~.program.sleep_coaching_mission = Counting sleep coaching program. 
* ~.program.sleep_coaching.session = Not sure.
* ~.report = Not sure.
* ~.rewards = Not sure.
* ~.sleep = Data collected during sleep. 
* ~.sleep_combined = Data collected during sleep. (Inaccurate)
* ~.sleep_goal = Not sure.
* ~.sleep_snoring = Not sure. Maybe information about snoring during sleep. Stored in samsung health application.
* ~.social.public_challenge.extra = Not sure.
* ~.social.service_status = Not sure.
* ~.stand_day_summary = Not sure.
* ~.step_dailty_trend = Not sure.
* ~.stress = Daily stress tracking value. 
* ~.stress.histogram = Not sure.
* ~.tracker.heart_rate = Daily heart rate tracking value. Contains the stress score between 1 and 100.
* ~.tracker.oxygen_saturation = Oxygen saturation value. Measured only when user activates.
* ~.tracker.pedometer_day_summary = Daily step counts, based on pedometer. But data must be organized by date.
* ~.tracker.pedometer_recommendation = Not sure.
* ~.tracker.pedometer_step_count = Daily step counts, based on pedometer. But data must be organized by date. 
* /files/ecg = pdf file format of ecg measures. Shows the 30seconds ecg results.

## Data file list that we need for Analysis

* ecg (both numerical csv file and pdf file) -- 
* activity_day_summary 
* exercise (maybe. for tracking what kind of exercise user did and for how long)
* sleep 
* stress
* stress.histogram (maybe. check json file)
* heart_rate
* oxygen saturation (maybe. have to search for more references)
* pedometer_day_summary (have to check the columns)
* tracker_pedometer_step_count

## Extracting data that we need

In [ ]:
import json
import os
import os.path
import sqlite3
import numpy as np
import pandas as pd

In [ ]:
from functools import reduce
from glob import glob

In [ ]:
# First setting the basic paths for data files
global csv_paths
wearable_path = 'E:/RESEARCH/wearable/' #adjust wearable data path

In [ ]:
# Exercise type code - 'exercise_type' column
exercise_type = {
    1001: 'walking',
    1002: 'running',
    14001: 'Swimming',
    0: 'other'
}

In [ ]:
# Defini tion for converting the data collected time.
def conv_date(x, col_name=''):
    try:
        if any('day_time' in col for col in x.columns):
            col_name = 'day_time'
            x['day_time'] = pd.to_datetime(
                x[[col for col in x.columns if col_name in col][0]], unit='ms')
        if any('start_time' in col for col in x.columns):
            col_name = 'start_time'
            x['day_time'] = pd.to_datetime(
                x[[col for col in x.columns if col_name in col][0]])
        x['day_time'] = pd.to_datetime(x['day_time'])
        
    except RuntimeError as e:
        print(e)
        print("Can't find one of these columns")
        print("Column names", x.columns)

In [ ]:
# Galaxy watch data contains json files. Definition for calling json files.
def get_json_file(s):
    join = os.path.join('**', s + '*.json')
    json_paths = glob(join, recursive=True)
    json_file = open(json_paths[0])
    return json.load(json_file)

In [ ]:
# mergeing data frame csv files that we need for the analysis
def df_meged():
    global csv_paths
    
    selected_csv = [
        'com.samsung.shealth.activity.day_summary'
    ]

In [ ]:
# extracting heart rate data (most important)
def heart_rate_data():
    """
    heart_rate_data reads json format heart rate data from wearable device,
    then return the extracted results as hr_df.
    
    dataframe contains heart rate, maximum heart rate, minimum heart rate, and measured time variables.
    """
    hr_path = os.path.join(wearable_path, '**', 'com.samsung.shealth.tracker.heart_rate','**','*.json')
    hr_files_path = glob(hr_path, recursive=True)
    hr_df = pd.DataFrame(columns=['heart_rate', 'heart_rate_max', 'heart_rate_min', 'start_time', 'end_time'])
    hr_df = hr_df.fillna(0) #filling NA/NAN values
    
    for f in hr_file_paths:
        try:
            with open(f) as file:
                json_hr = json.load(file)
                part_df = pd.DataFrame(json_hr)
                hr_df = pd.concat([hr_df, part_df])

        except ValueError as e:
            print(e)
    
    hr_df['day_time'] = pd.to_datetime(hr_df['start_time'], unit='ms')
    return hr_df

In [ ]:
# Extracting heart rate data by hourly time period
def hourly_heart_rate():
    hr_df = heart_rate_data()
    times = pd.DatetimeIndex(hr_df.day_time)
    hr_df = hr_df.groupby([times.hour, times.date]).agg('mean')
    hr_df.reset_index(inplace=True)
    hr_df['day_time'] = pd.to_datetime(
        pd.to_datetime(hr_df['level_1']).dt.strftime('%Y-%m-%d') + ' ' + hr_df['day_time'].astype(str) + ':00')

    return hr_df

## Extracting Final Data

In [ ]:
def data_extraction():
    global csv_paths
    base_dir = os.path.join(wearable_path)
    
    dump_dirs = glob(os.path.join(base_dir, '*'))
    dump_dir = os.path.basename(dump_dirs[0])
    print(len(dump_dirs), 'dumps found, taking first:', dump_dir)

    csv_paths = glob(os.path.join(base_dir, dump_dir, '*.csv'))
    print(len(csv_paths), 'csvs found')
    
    hr_data = hr_raw().sort_values(by='day_time', ascending=True)
    exercise_data = get_exercise_data()
    
    df_data_list = [hr_data]
    df_name_list = ['hr_data']

    for i, d in enumerate(df_data_list):
        try:
            d.to_csv('./' + str(df_name_list[i]) + '.csv')
        except Exception as e:
            print(e)
            
if __name__ == "__main__":
    data_extraction()